# loggerについて  
[loggerの参考URL①](https://www.sejuku.net/blog/23149)  
[loggerの考え方②](https://qiita.com/amedama/items/b856b2f30c2f38665701)  
[標準出力のログが重複する際の直し方参考URL](https://stackoverflow.com/questions/7173033/duplicate-log-output-when-using-python-logging-module)  
上記考え方を鑑みて次のセルの書き方に落ち着いた。  
解説は下段のセルを参照すること。

In [ ]:
# mylogging.py
from logging import getLogger, Formatter, StreamHandler, FileHandler, DEBUG

loggers = {}

def mylogger(name=None, myfmt='%(asctime)s %(name)s %(lineno)d %(levelname)s %(message)s', shl='INFO', fpath='result_tmp/input.log', fhl='DEBUG'):
	
	global loggers
	if name is None:
		name = __name__

	if loggers.get(name):
		return loggers.get(name)  #既出のロガーなら、ここで関数call終了

	# ログのフォーマットを設定
	fmt = Formatter(myfmt)

	# ログのコンソール出力の設定
	shandler = StreamHandler()
	shandler.setLevel(shl)
	shandler.setFormatter(fmt)

	# ログのファイル出力先の設定
	fhandler = FileHandler(fpath)
	fhandler.setLevel(fhl)
	fhandler.setFormatter(fmt)

	# ロガーの作成 
	logger = getLogger(__name__)  # ログの出力名を設定
	logger.setLevel(DEBUG)  #ログレベルの設定
	logger.addHandler(shandler)
	logger.addHandler(fhandler)
	logger.propagate = False
	loggers[name] = logger
	return logger


if __name__ == '__main__':
    #test = mylogger()

### ↓直接書く場合

In [ ]:
from logging import getLogger, Formatter, StreamHandler, FileHandler, DEBUG

# ログのフォーマットを設定
fmt = Formatter('%(asctime)s %(name)s %(lineno)d %(levelname)s %(message)s')

# ログのコンソール出力の設定
shandler = StreamHandler()
shandler.setLevel('INFO')
shandler.setFormatter(fmt)

# ログのファイル出力先の設定
fhandler = FileHandler('result_tmp/hoge.log')
fhandler.setLevel(DEBUG)
fhandler.setFormatter(fmt)

# ロガーの作成 
logger = getLogger(__name__)  # ログの出力名を設定
logger.setLevel(DEBUG)  #ログレベルの設定
logger.addHandler(shandler)
logger.addHandler(fhandler)
logger.propagate = False

# 実際に書く際は、以下のように処理中に記載してログを取る。
logger.info('start')
#df = pd.read_csv('input/train.csv')
logger.info('end')

### loggerについて解説  
[loggerの参考URL①](https://www.sejuku.net/blog/23149)  
[loggerの考え方②](https://qiita.com/amedama/items/b856b2f30c2f38665701)  
[標準出力のログが重複する際の直し方参考URL](https://stackoverflow.com/questions/7173033/duplicate-log-output-when-using-python-logging-module)

なお、pythonのロガーは木構造的になっていて、ロガーのクラス階層については以下のイメージ。
1. 根：Logging
1. 枝：Logger
1. 葉：LogRecord

結論として、根っこのloggingでログを書くとそのデフォルト設定が全ログに適用され、  
本来の木構造を活かすことができなくなるため（e.g. logging.debug()などは）避けるべき。  
例えば、logging.basicConfig(level='DEBUG')などと書くと、  
全てのログ出力がDEBUG以上になるのである。  
グローバル変数であるloggingの変更を全てのロガーに反映させないためにも、  
getLogger（'プロセス名'）を使って、各枝のロガークラスを各プロセスに割り当てるのが望ましい。

### ログレベル
|レベル|値|意味|
|:-:|:-:|:-:|
|debug|10|動作確認|
|info|20|正常動作|
|warn|30|ログの定義名|
|error|40|エラーなどの重大な問題|
|critical|50|停止などの致命的な問題|
なお、デフォルトではdebugとinfoは出力しないように設定されている。

### ログのFormatterの書き方
|フォーマット|役割|
|:-:|:-:|
|%(asctime)s|	実行時刻|
|%(filename)s|	ファイル名|
|%(funcName)s|	行番号|
|%(levelname)s|	ログの定義|
|%(lineno)d|	ログレベル名|
|%(message)s|	ログメッセージ|
|%(module)s|	モジュール名|
|%(name)s|	関数名|
|%(process)d|	プロセスID|
|%(thread)d|	スレッドID|

In [ ]:
# 解説用

# loggingパッケージから関数getLoggerなどをimportする。
from logging import getLogger, Formatter, StreamHandler, FileHandler, DEBUG

# ログのフォーマットを設定（詳細は次のマークダウンテーブル参考）
fmt = Formatter('%(asctime)s %(name)s %(lineno)d %(levelname)s %(message)s')

# ログのコンソール出力の設定
# 気を付けなければならないのは、shandler.setLevel()でINFOを指定していることである。
# そのため、logger.debug('処理')と書いた場合、INFOより下位レベルのdebugは表示されない。
shandler = StreamHandler()
shandler.setLevel('INFO')
shandler.setFormatter(fmt)

# ログのファイル出力先の設定
fhandler = FileHandler('result_tmp/hoge.log')
fhandler.setLevel(DEBUG)
fhandler.setFormatter(fmt)

# ログの出力名を設定
# __name__によって、直接実行しているmainファイルか、importされたファイルかを特定できる。
# つまり、どのファイルのログか特定できるため、ログを追いやすくなる。
logger = getLogger(__name__)
logger.setLevel(DEBUG)  # ログレベルの設定
logger.addHandler(shandler)
logger.addHandler(fhandler)
# logger.propagate = False

# 実際に書く際は、以下のように処理中に記載してログを取る。
logger.info('start')
#df = pd.read_csv('input/train.csv')
logger.info('end')